![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/6.Clinical_Context_Spell_Checker.ipynb)

<H1> Context Spell Checker - Medical </H1>


In [1]:
import json

with open('workshop_license_keys_Aug2020.json') as f:
    license_keys = json.load(f)

license_keys.keys()


dict_keys(['JSL_VERSION', 'PUBLIC_VERSION', 'SECRET', 'SPARK_NLP_LICENSE', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'JSL_OCR_SECRET', 'SPARK_OCR_LICENSE'])

In [2]:
license_keys['JSL_VERSION']

'2.5.4rc3'

In [ ]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

secret = license_keys['SECRET']

os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['SPARK_OCR_LICENSE'] = license_keys['SPARK_OCR_LICENSE']
os.environ['AWS_ACCESS_KEY_ID']= license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
jsl_version = license_keys['JSL_VERSION']
version = license_keys['PUBLIC_VERSION']

! pip install --ignore-installed -q pyspark==2.4.4

! python -m pip install --upgrade spark-nlp-jsl==$jsl_version  --extra-index-url https://pypi.johnsnowlabs.com/$secret

! pip install --ignore-installed -q spark-nlp==$version

import sparknlp

print (sparknlp.version())

import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

spark = sparknlp_jsl.start(secret)

In [4]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

tokenizer = RecursiveTokenizer()\
  .setInputCols(["document"])\
  .setOutputCol("token")\
  .setPrefixes(["\"", "(", "[", "\n"])\
  .setSuffixes([".", ",", "?", ")","!", "'s"])

spellModel = ContextSpellCheckerModel\
    .pretrained('spellcheck_clinical', 'en', 'clinical/models')\
    .setInputCols("token")\
    .setOutputCol("checked")

spellcheck_clinical download started this may take some time.
Approximate size to download 145 MB
[OK!]


In [6]:
pipeline = Pipeline(
    stages = [
    documentAssembler,
    tokenizer,
    spellModel
  ])

empty_ds = spark.createDataFrame([[""]]).toDF("text")

lp = LightPipeline(pipeline.fit(empty_ds))

Ok!, at this point we have our spell checking pipeline as expected. Let's see what we can do with it, see these errors,

_
__Witth__ the __hell__ of __phisical__ __terapy__ the patient was __imbulated__ and on posoperative, the __impatient__ tolerating a post __curgical__ soft diet._

_With __paint__ __wel__ controlled on __orall__ pain medications, she was discharged __too__ __reihabilitation__ __facilitay__._

_She is to also call the __ofice__ if she has any __ever__ greater than 101, or __leeding__ __form__ the surgical wounds._

_Abdomen is __sort__, nontender, and __nonintended__._

_Patient not showing pain or any __wealth__ problems._
            
_No __cute__ distress_

Check that some of the errors are valid English words, only by considering the context the right choice can be made.

In [7]:
example = ["Witth the hell of phisical terapy the patient was imbulated and on posoperative, the impatient tolerating a post curgical soft diet.",
            "With paint wel controlled on orall pain medications, she was discharged too reihabilitation facilitay.",
            "She is to also call the ofice if she has any ever greater than 101, or leeding form the surgical wounds.",
            "Abdomen is sort, nontender, and nonintended.",
            "Patient not showing pain or any wealth problems.",
            "No cute distress"
            
]

for pairs in lp.annotate(example):

  print (list(zip(pairs['token'],pairs['checked'])))

[('Witth', 'With'), ('the', 'the'), ('hell', 'cell'), ('of', 'of'), ('phisical', 'physical'), ('terapy', 'therapy'), ('the', 'the'), ('patient', 'patient'), ('was', 'was'), ('imbulated', 'ambulated'), ('and', 'and'), ('on', 'on'), ('posoperative', 'postoperative'), (',', ','), ('the', 'the'), ('impatient', 'patient'), ('tolerating', 'tolerating'), ('a', 'a'), ('post', 'post'), ('curgical', 'surgical'), ('soft', 'soft'), ('diet', 'diet'), ('.', '.')]
[('With', 'With'), ('paint', 'pain'), ('wel', 'well'), ('controlled', 'controlled'), ('on', 'on'), ('orall', 'oral'), ('pain', 'pain'), ('medications', 'medications'), (',', ','), ('she', 'she'), ('was', 'was'), ('discharged', 'discharged'), ('too', 'to'), ('reihabilitation', 'rehabilitation'), ('facilitay', 'facility'), ('.', '.')]
[('She', 'She'), ('is', 'is'), ('to', 'to'), ('also', 'also'), ('call', 'call'), ('the', 'the'), ('ofice', 'once'), ('if', 'if'), ('she', 'she'), ('has', 'has'), ('any', 'any'), ('ever', 'fever'), ('greater', 'g